In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

KeyboardInterrupt: 

In [3]:
data = pd.read_excel('filtered_data_3.xlsx')

In [4]:
data['combined_text'] = data['combined_text'].fillna('')

In [5]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['combined_text'])

In [6]:
print("TF-IDF Matrix shape:", tfidf_matrix.shape)

TF-IDF Matrix shape: (210, 132)


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular a similaridade do cosseno
cosine_similarities = cosine_similarity(tfidf_matrix)

# Mostrar a matriz de similaridade para inspecionar
print("Cosine Similarities shape:", cosine_similarities.shape)

Cosine Similarities shape: (210, 210)


In [11]:

# Função para recomendar métricas e calcular a porcentagem de similaridade
def recommend_metrics_with_similarity(manager_id, cosine_similarities, data, top_n=5):
    similar_indices = cosine_similarities[manager_id].argsort()[:-top_n-1:-1]
    similar_managers = data.iloc[similar_indices]
    
    # Extrair métricas utilizadas pelos gerentes similares
    metrics_columns = [col for col in data.columns if col.startswith('metric_')]
    
    metrics_dict = {}
    for index in similar_indices:
        similarity_score = cosine_similarities[manager_id][index]
        manager_metrics = data.iloc[index][metrics_columns].dropna()
        
        for metric in manager_metrics:
            if metric not in metrics_dict:
                metrics_dict[metric] = 0
            metrics_dict[metric] += similarity_score

    # Calcular a porcentagem de similaridade para cada métrica
    total_similarity = sum(metrics_dict.values())
    metrics_similarity_percentage = {metric: (similarity / total_similarity) * 100 for metric, similarity in metrics_dict.items()}
    
    return metrics_similarity_percentage

In [12]:
# Exemplo de recomendação para o gerente com ID 0
manager_id = 0
recommended_metrics_with_similarity = recommend_metrics_with_similarity(manager_id, cosine_similarities, data)
print("Recommended metrics with similarity percentages for manager ID", manager_id, ":", recommended_metrics_with_similarity)

Recommended metrics with similarity percentages for manager ID 0 : {'Ideal team capacity': np.float64(19.231024440050245), 'CFD': np.float64(8.974330617772194), 'Burndown': np.float64(8.974330617772194), 'CAC': np.float64(8.974330617772194), 'Roadmaps': np.float64(8.974330617772194), 'Throughput': np.float64(8.974330617772194), 'Lead time': np.float64(8.974330617772194), 'Conversão': np.float64(8.974330617772194), 'NPS': np.float64(8.974330617772194), 'Team total available hours': np.float64(8.974330617772194)}
